# first of all go to playstore and install the tello app and then connect it with your wifi

In [ ]:
!pip install djitellopy

One thing is remember that tello app connected to your wifi

In [ ]:
from djitellopy import tello
from time import sleep
me=tello.Tello() # tello object
me.connect()
print(me.get_battery()) # how many battery remains in my drone

In [ ]:
# move foreard means tello moves x cm forward
me.move_forward(30) # 30 cm
# we wants to control that becoz drone move 3 directions and rotate in one directions
me.takeoff()
me.send_rc_control(0,50,0,0)# left right vellocity (left_right_vellocity,fireard_backward_vellocity,up_down_velocity,yaw_velocity)
# and return True and False for successfull or not 
sleep(2) # speed of 2 second 50 after t
me.send_rc_control(0,0,0,0)
sleep(2)
me.send_rc_control(0,0,0,0)
me.land() # after come to on the ground

## Image Capturing

In [ ]:
# Stream gives the all the frame and process that 
import cv2
me.streamon()
while True:
  image=me.get_frame_read().frame # that will be gives the individual image comming from the tellp drone
  # if we wants to process the image faster then we needs to process the images faster by using the open cv library
  image=cv2.resize(image,(360,240))
  cv2.imshow("Image",image)
  cv2.waitkey(1) # gives the delay 1 ms of the image

##keypress.py

In [ ]:
import pygame

# create the window
def init():
  pygame.init()
  win=pygame.display.set_mode((400,400)) # gives the size

# get the key press
def getkey(keyName): # represent wether the key pressed or not
  ans=False
  for eve in pygame.event.get():pass
  # now check the true and flase and create the certian formate
  keyInput=pygame.key.get_pressed()
  mykey=getattr(pygame,"k_{}".format(keyName))
  if keyInput[mykey]:
    ans=True
  pygame.display.update()
  return ans

def main():
  # print(getkey("a"))# if you press key (a) then wether the code compile at that time
  # return " "
  if getkey("LEFT"):
    print("Left Key pressed")
  elif getkey("RIGHT"):
    PRINT("right key pressed")
  else:
    print(" ")


if __name__=="__main__":
  init()
  while True:
    main()
    


## keyword Control

now we have used to the keyword for controlling the our drone and after then we have implements the basic movementem for that we have create new module that called key press module

In [ ]:
from djitellopy import tello
from time import sleep
import KeyPressModule as kp
import cv2
import time
kp.init()
me=tello.Tello()
me.connect()
print(me.get_battery())

def getkeyboardInput():
  lr,fb,upyv=0,0,0,0
  if kp.getkey("LEFT"):
    lr=-speed
  elif kp.getkey("RIGHT"):
    lr=speed

  if kp.getkey("UP"):
    fb=-speed
  elif kp.getkey("DOWN"):
    fb=speed

  if kp.getkey("w"):
    ud=-speed
  elif kp.getkey("s"):
    ud=speed

  if kp.getkey("a"):
    yv=-speed
  elif kp.getkey("d"):
    yv=speed
  
  if kp.getkey("q"):
    yl=me.land()
    time.sleep(3)
  if kp.getkey("e"):
    yl=me.takeoff()

  if kp.getkey("z"):
    cv2.imwrite(f"Resources/Images/{time.time()}.jpg")

  return [lr,fb,ud,yv]
me.takeoff()

while True:
  vals=getkeyboardInput()
  me.send_rc_control(vals[0],vals[1],vals[2],vals[3])
  image=me.get_frame_read().frame # that will be gives the individual image comming from the tellp drone
  # if we wants to process the image faster then we needs to process the images faster by using the open cv library
  image=cv2.resize(image,(360,240))
  cv2.imshow("Image",image)
  cv2.waitkey(1) # gives the delay 1 ms of the image


## Mapping

Every steps we have needs to calulate the cordinate 
* x=d*cosine(45) ,x cordinate
* y=d*cosine(45) ,y cordinate





In [ ]:
from djitellopy import tello
from time import sleep
import KeyPressModule as kp
import cv2
import time
import  numpy as np
import math
###################################### INITALIZE THE PARAMETERS #################################
fspeed=117/10 # forward speed in cm/s  (15 cm/s)
aspeed=360/10 # Angular Speed in Degree/s
interval=0.25

dInterval=fspeed*interval
aInterval=aspeed*interval
######################################################################3

# let's find the values of for we have travel so idea is that at any other points we have plot to the x and y
x,y=500
# and angel that can't not be zero that will be continue by distance will be reset every time
kp.init()
me=tello.Tello()
me.connect()
print(me.get_battery())
points=[(0,0),(0,0)] # that is not empty in bignning we have 2 values other wise gives you error index is out of range 
yaw=0
def getkeyboardInput():
  lr,fb,upyv=0,0,0,0
  global yaw,x,y,a
  speed=15
  aspeed=50
  if kp.getkey("LEFT"):
    lr=-speed
    d=dInterval
    # we will not travelling forward directions we will travelling towards the west and we have change our angel
    a=-180
  elif kp.getkey("RIGHT"):
    lr=speed
    d=-dInterval
    # we will not travelling forward directions we will travelling towards the west and we have change our angel
    a=180

  if kp.getkey("UP"):
    fb=-speed
    d=dInterval
    # we will not travelling forward directions we will travelling towards the west and we have change our angel
    a=270
  elif kp.getkey("DOWN"):
    fb=-speed
    d=-dInterval
    # we will not travelling forward directions we will travelling towards the west and we have change our angel
    a=-90

  if kp.getkey("w"):
    ud=-speed
  elif kp.getkey("s"):
    ud=speed

  if kp.getkey("a"):
    yv=-speed
    yaw-=aInterval
  elif kp.getkey("d"):
    yv=speed
    yaw+=aInterval
  
  if kp.getkey("q"):
    yl=me.land()
    time.sleep(3)
  if kp.getkey("e"):
    yl=me.takeoff()

  if kp.getkey("z"):
    cv2.imwrite(f"Resources/Images/{time.time()}.jpg")
  
  # update the cartision product
  time.sleep(0.25)
  a+=yaw
  x+=int(d*math.cos(math.radians(a)))
  y+=int(d*math.sin(math.radians(a)))

  return [lr,fb,ud,yv]
me.takeoff()

def drowPoints(img,points):
  # cv2.circle(img,(300,500),10,(0,0,255),cv2.FILLED)
  for point in points:
    cv2.circle(img,points[-1],8,(0,255,0),cv2.FILLED) # green colours

  cv2.putText(img,f"({points[-1][0]-250},{(points[-1][1]-500)/100})m",(points[-1][0]+10,points[-1][1]+30,# becoz ur first positions is 500
                                                                     cv2.FONT_HERSHEY_PLAIN,1,# 1=thickness
                                                                    (255,0,255),1 )
  return " "

while True:
  vals=getkeyboardInput()
  me.send_rc_control(vals[0],vals[1],vals[2],vals[3])
  sleep(0.05)
  img=np.zeros((1000,1000,3),np.uint8)
  points.append((vals[4],vals[5]))
  if points[-1][0]!=vals[4] or points[-1][1]!=vals[5]: # that means we are not plotting the same values again by again
    points.append(vals[4],vals[5])
    
  drowPoints(img,points)
  cv2.imshow("Output",img)
  cv2.waitkey(1)

## Face Tracking



* If Face Area is small then-----> Move Forward

* If Face Area is Big then------> Move Backward




In [ ]:
import numpy as np
import cv2
pid=[0.4,0.4,0
     ]
fbRange=[6200,6800] # forward range
def FindFace(img):
  faceCascade=cv2.CascadeClassifier("##############XYZ####################") # txt file
  imgGray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  faces=faceCascade.detectMultiScale(imgGray,1.2,8)
  # Then we are going to create the list of all over the faces that are lot of faces we have to find out the bigger one
  myFaceList=[]
  myFaceListArea=[]

  for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2) # starting and ending points with thickness 2 and color red
    cx=x+w//2
    cy=y+h//2
    area=w+h
    cv2.circle(img,(cx,cy),5,(0,255,0),cv2.FILLED)# radius =5
    myFaceList.append([cx,cy])
    myFaceListArea.append(area)
  if len(myFaceListArea)!=0:
    i=myFaceListArea.index(max(myFaceListArea))
    return img,[myFaceList[i],myFaceListArea[i]]
  else:
    return img,[[0,0],0]
  
def trackface(me,info,w,pid,pError): # w=width,
  area=info[1]
  x,y=info[0]
  error=x-w//2
  speed=pid[0]
  if area>fbRange[0] and area<fbRange[1]:
    fb=0
  elif area>fbRange[2]: # if it is too close or too big then go backward
    fb=-20
  elif area<fbRange[0] and area!=0: # if it is too or too small far then go forward
    fb=20
  else:  
    return " "             

cap=cv2.VideoCapture(1)
while True:
  _,img=cap.read()
  img,info=FindFace(img)
  print("Area",info[1],"##","Center",info[0])
  cv2.imshow("Output",img)
  cv2.waitKey(1)


